# 1.Temperature Analysis and Visualization Near Ann Arbor, Michigan

* This notebook analyzes temperature data from weather stations near Ann Arbor, Michigan.

* The analysis includes:

1. Visualizing record high and low temperatures (2005-2014) with 2015 overlaid.

2. Plotting weather station locations on a map.

3. Summarizing 2015 temperature data by month.

---

#2. Import Libraries

In [51]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

**Note:**

* Pandas: For data manipulation and analysis.

* NumPy: For numerical operations.

* Plotly: For interactive visualizations (chosen over Matplotlib for better interactivity and aesthetics).

# 3. Load Datasets

In [52]:
temp = pd.read_csv('temperature.csv')

bin = pd.read_csv('BinSize.csv')

# 3. Data Preprocessing

In [53]:
# Convert Date to datetime
temp['Date'] = pd.to_datetime(temp['Date'])

# Filter out leap days (February 29th)
temp = temp[~((temp['Date'].dt.month == 2) & (temp['Date'].dt.day == 29))]

# Extract year and day of year
temp['Year'] = temp['Date'].dt.year
temp['DayOfYear'] = temp['Date'].dt.dayofyear

<ipython-input-53-274b932118e0>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-53-274b932118e0>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



**Note:**

* Leap days are removed to simplify the analysis.

* Additional columns (`Year` and `DayOfYear`) are created for grouping and visualization.


---

# 5. Record High and Low Temperatures (2005-2014) with 2015 Overlay

In [56]:
# Filter data for 2005-2014 and 2015
temp_2005_2014 = temp[(temp['Year'] >= 2005) & (temp['Year'] <= 2014)]
temp_2015 = temp[temp['Year'] == 2015]

# Group by day of year and find record high and low temperatures
record_high = temp_2005_2014[temp_2005_2014['Element'] == 'TMAX'].groupby('DayOfYear')['Data_Value'].max()
record_low = temp_2005_2014[temp_2005_2014['Element'] == 'TMIN'].groupby('DayOfYear')['Data_Value'].min()

# Find 2015 points where records were broken
highs_2015 = temp_2015[temp_2015['Element'] == 'TMAX']
lows_2015 = temp_2015[temp_2015['Element'] == 'TMIN']

highs_broken = highs_2015[highs_2015.apply(lambda x: x['Data_Value'] > record_high[x['DayOfYear']], axis=1)]
lows_broken = lows_2015[lows_2015.apply(lambda x: x['Data_Value'] < record_low[x['DayOfYear']], axis=1)]

fig = go.Figure()

fig.add_trace(go.Scatter(x=record_high.index, y=record_high.values, mode='lines', name='Record High (2005-2014)', line=dict(color='red')))
fig.add_trace(go.Scatter(x=record_low.index, y=record_low.values, mode='lines', name='Record Low (2005-2014)', line=dict(color='blue')))

fig.add_trace(go.Scatter(x=np.concatenate([record_high.index, record_low.index[::-1]]), y=np.concatenate([record_high.values, record_low.values[::-1]]), fill='toself', fillcolor='rgba(128, 128, 128, 0.2)', line=dict(color='rgba(255, 255, 255, 0)'), name='Range between High and Low'))

fig.add_trace(go.Scatter(x=highs_broken['DayOfYear'], y=highs_broken['Data_Value'], mode='markers', name='2015 Highs Breaking Records', marker=dict(color='darkred', size=8)))
fig.add_trace(go.Scatter(x=lows_broken['DayOfYear'], y=lows_broken['Data_Value'], mode='markers', name='2015 Lows Breaking Records', marker=dict(color='darkblue', size=8)))

fig.update_layout(title='Record High and Low Temperatures (2005-2014) with 2015 Overlay', xaxis_title='Day of Year', yaxis_title='Temperature (°C)', hovermode='x unified', legend=dict(x=0.02, y=0.98), template='plotly_white')

fig.show()

**Note:**

* The visualization shows record high and low temperatures (2005-2014) with 2015 data overlaid.

* Plotly is used for interactivity (hover, zoom, pan).

---

# 6. Visualize Stations on a Map

In [57]:
# Filter stations near Ann Arbor, Michigan
ann_arbor_lat, ann_arbor_lon = 42.28, -83.74
stations_near_ann_arbor = bin[(bin['LATITUDE'].between(ann_arbor_lat - 1, ann_arbor_lat + 1)) & (bin['LONGITUDE'].between(ann_arbor_lon - 1, ann_arbor_lon + 1))]

fig_map = px.scatter_mapbox(stations_near_ann_arbor, lat='LATITUDE', lon='LONGITUDE', hover_name='NAME', hover_data=['STATE', 'ELEVATION'], zoom=8, height=600, title='Weather Stations Near Ann Arbor, Michigan', size_max=15, color_discrete_sequence=['red'])

fig_map.update_traces(marker=dict(size=12, opacity=0.8))

fig_map.update_layout(mapbox_style="open-street-map", margin={"r": 0, "t": 40, "l": 0, "b": 0})

fig_map.show()

**Note:**

* Weather stations near Ann Arbor are plotted on an interactive map.

* Points are larger and colored red for better visibility.

# 7.  Plot Temperature Summary for 2015

In [61]:
# Filter 2015 data
temp_2015 = temp[temp['Year'] == 2015]

# Extract month from the Date column
temp_2015['Month'] = temp_2015['Date'].dt.month

# Group by month and calculate average temperature
monthly_avg_temp = temp_2015.groupby('Month')['Data_Value'].mean().reset_index()

fig_temp_summary = go.Figure()

fig_temp_summary.add_trace(go.Bar(x=monthly_avg_temp['Month'], y=monthly_avg_temp['Data_Value'], name='Monthly Average Temperature', marker_color='lightblue'))

fig_temp_summary.update_layout(title='Monthly Average Temperature Near Ann Arbor, Michigan (2015)', xaxis_title='Month', yaxis_title='Temperature (°C)', xaxis=dict(tickmode='array', tickvals=list(range(1, 13)), ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']), template='plotly_white')

fig_temp_summary.show()

<ipython-input-61-ccc6af42f36e>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



**Note:**

* A bar chart shows the monthly average temperatures for 2015.

* The x-axis displays month names for better readability.

---
# 8. Conclusion
* Summarize the findings:

 * Record high and low temperatures (2005-2014) with 2015 overlaid.

 * Locations of weather stations near Ann Arbor.

 * Monthly temperature trends for 2015.


---